In [1]:
import numpy as np
import pandas as pd
import sklearn
import time
import os
from data import *
from models import *
from simulation import *

In [3]:
seedResults = createSeedResultsDF(seeds, tourneyCompactResults)

matchups = [generateMatchups(1), generateMatchups(2), generateMatchups(3), generateMatchups(4)]

In [5]:
print('')
print(regSeasDetailedTotals.columns)


Index(['Season', 'TeamID', 'G', 'W', 'L', 'Pts', 'PA', 'FGM', 'FGA', 'FGM3',
       'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF'],
      dtype='object')


In [12]:
def createMasterDetailedDF(seedResults, regSeasDetailedTotals):
    srFn = seedResults[:] #create copy to avoid modifying original
    rstFn = regSeasDetailedTotals[:]

    rstFn.rename(columns = {'TeamID': 'WTeamID'}, inplace = True) #this procedure is the same as in createSeedResultsDF function above
    outDF = pd.merge(srFn, rstFn, on = ['WTeamID', 'Season'])

    outDF.rename(columns = {'G': 'WG','W': 'WWins', 'L': 'WLosses', 'Pts': 'WPts', 'PA': 'WPA', 'FGM': 'WFGM', 'FGA': 'WFGA', 'FGM3': 'WFGM3',
       'FGA3': 'WFGA3', 'FTM': 'WFTM', 'FTA': 'WFTA', 'OR': 'WOR', 'DR': 'WDR', 'Ast': 'WAst', 'TO': 'WTO', 'Stl': 'WStl', 'Blk': 'WBlk', 'PF': 'WPF'}, inplace = True)

    rstFn.rename(columns = {'WTeamID': 'LTeamID'}, inplace = True)
    outDF = pd.merge(outDF, rstFn, on = ['LTeamID', 'Season'])

    outDF.rename(columns = {'G': 'LG','W': 'LWins', 'L': 'LLosses', 'Pts': 'LPts', 'PA': 'LPA', 'FGM': 'LFGM', 'FGA': 'LFGA', 'FGM3': 'LFGM3',
       'FGA3': 'LFGA3', 'FTM': 'LFTM', 'FTA': 'LFTA', 'OR': 'LOR', 'DR': 'LDR', 'Ast': 'LAst', 'TO': 'LTO', 'Stl': 'LStl', 'Blk': 'LBlk', 'PF': 'LPF'}, inplace = True)

    return outDF

In [14]:
jack = createMasterDetailedDF(seedResults, regSeasDetailedTotals)
jack.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WSection', 'WNumSeed', 'LSection', 'LNumSeed', 'WG', 'WWins',
       'WLosses', 'WPts', 'WPA', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM',
       'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LG',
       'LWins', 'LLosses', 'LPts', 'LPA', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [ ]:
def createMasterCompactDF(seedResults, regSeasCompactTotals):
    srFn = seedResults[:] #create copy to avoid modifying original
    rstFn = regSeasCompactTotals[:]

    rstFn.rename(columns = {'TeamID': 'WTeamID'}, inplace = True) #this procedure is the same as in createSeedResultsDF function above
    outDF = pd.merge(srFn, rstFn, on = ['WTeamID', 'Season'])
    outDF.rename(columns = {'G': 'WG','W': 'WWins', 'L': 'WLosses', 'Pts': 'WPts', 'PA': 'WPA'}, inplace = True)

    rstFn.rename(columns = {'WTeamID': 'LTeamID'}, inplace = True)
    outDF = pd.merge(outDF, rstFn, on = ['LTeamID', 'Season']) 
    outDF.rename(columns = {'G': 'LG','W': 'LWins', 'L': 'LLosses', 'Pts': 'LPts', 'PA': 'LPA'}, inplace = True)

    return outDF

In [10]:
print('')
print(master1985.head())


  Season  DayNum WTeamID  WScore  LTeamID  LScore WLoc  NumOT WSection  \
0   1985     136    1116      63     1234      54    N      0        X   
1   1985     136    1120      59     1345      58    N      0        Z   
2   1985     138    1120      66     1242      64    N      0        Z   
3   1985     136    1207      68     1250      43    N      0        W   
4   1985     138    1207      63     1396      46    N      0        W   

   WNumSeed  ...    WG  WWins  WLosses    WPts     WPA    LG  LWins  LLosses  \
0       9.0  ...  33.0   21.0     12.0  2156.0  2036.0  30.0   20.0     10.0   
1      11.0  ...  29.0   18.0     11.0  2040.0  1933.0  25.0   17.0      8.0   
2      11.0  ...  29.0   18.0     11.0  2040.0  1933.0  30.0   23.0      7.0   
3       1.0  ...  27.0   25.0      2.0  2045.0  1622.0  29.0   11.0     18.0   
4       1.0  ...  27.0   25.0      2.0  2045.0  1622.0  29.0   24.0      5.0   

     LPts     LPA  
0  2092.0  1778.0  
1  1728.0  1633.0  
2  2281.0  21

In [24]:
jack = tourneySim(2019, highSeedWins, master1985)
karisch = tourneyActual(2019, master1985)


In [25]:
accuracyVector = [x == y for x, y in zip(jack, karisch)]
accuracy = sum(accuracyVector) / len(accuracyVector)

print(accuracy)

for i in range(len(jack)):
    print('Guess: ', jack[i], ' Actual: ', karisch[i])

0.6865671641791045
Guess:  1181  Actual:  1181
Guess:  1433  Actual:  1416
Guess:  1280  Actual:  1251
Guess:  1439  Actual:  1439
Guess:  1268  Actual:  1268
Guess:  1261  Actual:  1261
Guess:  1257  Actual:  1278
Guess:  1277  Actual:  1277
Guess:  1211  Actual:  1211
Guess:  1393  Actual:  1124
Guess:  1266  Actual:  1293
Guess:  1199  Actual:  1199
Guess:  1138  Actual:  1138
Guess:  1403  Actual:  1403
Guess:  1305  Actual:  1196
Guess:  1276  Actual:  1276
Guess:  1314  Actual:  1314
Guess:  1429  Actual:  1449
Guess:  1120  Actual:  1120
Guess:  1242  Actual:  1242
Guess:  1235  Actual:  1326
Guess:  1222  Actual:  1222
Guess:  1459  Actual:  1459
Guess:  1246  Actual:  1246
Guess:  1438  Actual:  1438
Guess:  1279  Actual:  1328
Guess:  1458  Actual:  1332
Guess:  1243  Actual:  1414
Guess:  1437  Actual:  1437
Guess:  1345  Actual:  1345
Guess:  1153  Actual:  1234
Guess:  1397  Actual:  1397
Guess:  1181  Actual:  1181
Guess:  1439  Actual:  1439
Guess:  1261  Actual:  1261
G

In [20]:
sum(accuracyVector)

42

In [70]:
jack = [[1, 2], 3, [4, 5], [6, 7, 8]]


"""
        winWBool = nonPlayInGames.loc[i, 'WSection'] == 'W'
        winXBool = nonPlayInGames.loc[i, 'WSection'] == 'X'
        loseWBool = nonPlayInGames.loc[i, 'LSection'] == 'W'
        loseXBool = nonPlayInGames.loc[i, 'LSection'] == 'X'
        winYBool = nonPlayInGames.loc[i, 'WSection'] == 'Y'
        winZBool = nonPlayInGames.loc[i, 'WSection'] == 'Z'
        loseYBool = nonPlayInGames.loc[i, 'LSection'] == 'Y'
        loseZBool = nonPlayInGames.loc[i, 'LSection'] == 'Z'
        if (winWBool and loseXBool) or (winXBool and loseWBool):
"""

print(reversed(jack))

In [62]:
print('')
print(master1985.head())
print(master1985.columns)
os.getcwd()
master1985.to_csv(path_or_buf = '../GeneratedData/master1985.csv')


  Season  DayNum WTeamID  WScore  LTeamID  LScore WLoc  NumOT WSection  \
0   1985     136    1116      63     1234      54    N      0        X   
1   1985     136    1120      59     1345      58    N      0        Z   
2   1985     138    1120      66     1242      64    N      0        Z   
3   1985     136    1207      68     1250      43    N      0        W   
4   1985     138    1207      63     1396      46    N      0        W   

   WNumSeed  ...    WG  WWins  WLosses    WPts     WPA    LG  LWins  LLosses  \
0       9.0  ...  33.0   21.0     12.0  2156.0  2036.0  30.0   20.0     10.0   
1      11.0  ...  29.0   18.0     11.0  2040.0  1933.0  25.0   17.0      8.0   
2      11.0  ...  29.0   18.0     11.0  2040.0  1933.0  30.0   23.0      7.0   
3       1.0  ...  27.0   25.0      2.0  2045.0  1622.0  29.0   11.0     18.0   
4       1.0  ...  27.0   25.0      2.0  2045.0  1622.0  29.0   24.0      5.0   

     LPts     LPA  
0  2092.0  1778.0  
1  1728.0  1633.0  
2  2281.0  21

In [10]:
class tourneyGame:
    def __init__(self, evalFn, priorGameA = None, priorGameB = None, teamA = None, teamB = None):
        self.evalFn = evalFn
        self.priorGameA = priorGameA
        self.priorGameB = priorGameB
        self.teamA = teamA
        self.teamB = teamB

    def findWinner(self):
        if self.teamA == None:
            self.teamA = self.priorGameA.findWinner()
        if self.teamB == None:
            self.teamB = self.priorGameB.findWinner()
        
        self.winner = self.evalFn(self.teamA, self.teamB)

        return self.winner

class tourneyClass:
    def __init__(self, evalFn):
        self.net = []
        for _ in range(32):
            self.net.append(tourneyGame(evalFn))
        count = 0
        for i in [16, 8, 4, 2, 1]:
            for _ in range(i):
                self.net.append(tourneyGame(evalFn, priorGameA = self.net[count], priorGameB = self.net[count + 1]))
                count += 2
    
    def simulate(self):
        self.net[-1].findWinner()
        return [x.winner for x in self.net]

In [56]:
jack = tourneyClass(testfn)

{'evalFn': <function __main__.testfn(a, b)>,
 'priorGameA': <__main__.tourneyGame at 0xa1ba01940>,
 'priorGameB': <__main__.tourneyGame at 0xa1ba01908>,
 'teamA': None,
 'teamB': None}

In [12]:
def testfn(a, b):
    return a + b

17

In [29]:
jack = []
jack.append(tourneyGame(evalFn = testfn, teamA = 5, teamB = 7))
jack.append(tourneyGame(evalFn = testfn, priorGameA = jack[0], teamB = 9))
jack.append(tourneyGame(evalFn = testfn, teamA = 3, teamB = 4))
jack.append(tourneyGame(evalFn = testfn, priorGameA = jack[1], priorGameB = jack[2]))

jack[3].findWinner()
jack[2].winner

7

In [8]:
seedResultsCompact = createSeedResults1985DF(seeds, tourneyCompactResults)
master1985 = createSeedResultRegTotals1985DF(seedResultsCompact, regSeasCompactTotals)

In [10]:
print('')
print(master1985[0:5])
print(master1985[master1985['Season'] == 1985].shape)
print(master1985.columns)


  Season  DayNum WTeamID  WScore  LTeamID  LScore WLoc  NumOT WSection  \
0   1985     136    1116      63     1234      54    N      0        X   
1   1985     136    1120      59     1345      58    N      0        Z   
2   1985     138    1120      66     1242      64    N      0        Z   
3   1985     136    1207      68     1250      43    N      0        W   
4   1985     138    1207      63     1396      46    N      0        W   

   WNumSeed  ...    WG  WWins  WLosses    WPts     WPA    LG  LWins  LLosses  \
0       9.0  ...  33.0   21.0     12.0  2156.0  2036.0  30.0   20.0     10.0   
1      11.0  ...  29.0   18.0     11.0  2040.0  1933.0  25.0   17.0      8.0   
2      11.0  ...  29.0   18.0     11.0  2040.0  1933.0  30.0   23.0      7.0   
3       1.0  ...  27.0   25.0      2.0  2045.0  1622.0  29.0   11.0     18.0   
4       1.0  ...  27.0   25.0      2.0  2045.0  1622.0  29.0   24.0      5.0   

     LPts     LPA  
0  2092.0  1778.0  
1  1728.0  1633.0  
2  2281.0  21

In [16]:
temp = ((1, 16), (8, 9), (5, 12), (4, 13), (6, 11), (3, 14), (7, 10), (2, 15))

temp2 = ((1, 8), (1, 9), (16, 8), (16, 9), (5, 4), (5, 13), (12, 4), (12, 13))

def generateGames(round):
    seedNums = [1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]

    out = []
    
    count = 0
    for i in range(2 ** round):
        groupSize = ((2 ** round) / 2)
        pivots = seedNums[count:(count + groupSize)]
        leaves = seedNums[(count + groupSize + 1):(count + groupSize + 1 + groupSize)]
        for j in pivots:
            for k in leaves:
                out.append([j, k])
    
    return out

generateGames(1)

TypeError: slice indices must be integers or None or have an __index__ method

In [14]:
jack = ((1, 2), (3, 4))
2 ** 3
jack[0:0]

()

In [ ]:
nrow = seedResults.shape[0]
seedResults['WExpected'] = seedResults.WTeamID

start = time.time()

for i in range(nrow):
    if seedResults.loc[i, 'WNumSeed'] >= seedResults.loc[i, 'LNumSeed']:
        seedResults.loc[i, 'WExpected'] = seedResults.loc[i, 'LTeamID']

end = time.time()

print(end - start)

seedResults['correctGuess'] = seedResults.WExpected == seedResults.WTeamID

In [ ]:
accuracy = seedResults.correctGuess.sum() / nrow
print("Picking highest seed strategy accuracy: ", accuracy)

In [6]:
seedResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WSection,WNumSeed,LSection,LNumSeed
0,1985,136,1116,63,1234,54,N,0,X,9.0,X,8.0
1,1985,136,1120,59,1345,58,N,0,Z,11.0,Z,6.0
2,1985,138,1120,66,1242,64,N,0,Z,11.0,Z,3.0
3,1985,136,1207,68,1250,43,N,0,W,1.0,W,16.0
4,1985,138,1207,63,1396,46,N,0,W,1.0,W,8.0


In [ ]:
def tourneyDriver(matchupsDF, referenceDF):
    matchups = matchupsDF[:]
    reference = referenceDF[:]